In [1]:
import os
os.chdir(r"C:\Users\PC\Desktop\work\projectML\kidney-disease-classification-MLflow")
%pwd

'C:\\Users\\PC\\Desktop\\work\\projectML\\kidney-disease-classification-MLflow'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    

    def download_file(self)->str:
        """
        Fetch data from the url
        
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-27 18:53:11,753: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 18:53:11,754: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 18:53:11,754: INFO: common: created directory at: artifacts]
[2025-03-27 18:53:11,755: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-27 18:53:11,756: INFO: 3434152109: Downloading data from https://drive.google.com/file/d/1itZiOjuG9E4KevkdhpfJhlKY1rr2jFbv/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1itZiOjuG9E4KevkdhpfJhlKY1rr2jFbv
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1itZiOjuG9E4KevkdhpfJhlKY1rr2jFbv&confirm=t&uuid=7222c9cf-e96e-4a24-8a0a-36bcef7e305e
To: C:\Users\PC\Desktop\work\projectML\kidney-disease-classification-MLflow\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:10<00:00, 5.61MB/s]

[2025-03-27 18:53:27,021: INFO: 3434152109: downloaded data from https://drive.google.com/file/d/1itZiOjuG9E4KevkdhpfJhlKY1rr2jFbv/view?usp=sharing into file artifacts/data_ingestion/data.zip]
